# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [10]:
#new
context_user = context.copy()
print(return_CCRMSQL("What is the average salary of employees by year?", context_user))

To calculate the average salary of employees by year, we need to have a table that includes the salary of employees and the year they received that salary. Let's create a table named "EmployeeSalary" with columns for employee ID, salary, and year.

```sql
CREATE TABLE EmployeeSalary (
    employee_id INT,
    salary DECIMAL(10, 2),
    year INT
);
```

Now, assuming we have data in the "EmployeeSalary" table, we can calculate the average salary of employees by year using the following query:

```sql
SELECT year, AVG(salary) AS average_salary
FROM EmployeeSalary
GROUP BY year;
```


In [11]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Which employees have a salary higher than 50000 in 2023?", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > 50000 AND s.year = '2023';
```

This SQL query selects the names of employees from the "employees" table who have a salary higher than 50000 in the year 2023 from the "salary" table by joining both tables on the employee ID.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [12]:
print(return_CCRMSQL("Show the names of employees who have a specialty in 'Engineering' and earn more than 60,000 in 2022?", context_user))

```sql
SELECT e.name
FROM employees e
JOIN specialties s ON e.specialty_id = s.id
JOIN salaries sal ON e.id = sal.employee_id
WHERE s.name = 'Engineering' AND sal.amount > 60000 AND sal.year = 2022;
```


In [13]:
print(return_CCRMSQL("How many employees have been employed for more than 5 years?", context_user))

```sql
SELECT COUNT(*) 
FROM employees
WHERE years_employed > 5;
```


In [14]:
print(return_CCRMSQL("List all employees who have a salary higher than the average salary for their educational level in 2023?", context_user))

```sql
WITH avg_salary_by_education AS (
    SELECT education_level, AVG(salary) AS avg_salary
    FROM employees
    WHERE year = 2023
    GROUP BY education_level
)

SELECT e.*
FROM employees e
JOIN avg_salary_by_education a ON e.education_level = a.education_level
WHERE e.salary > a.avg_salary AND e.year = 2023;
```
